In [ ]:
from __future__ import annotations

from typing import cast

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from modelbase2 import LabelMapper, LinearLabelMapper, Model, Simulator, plot


def mass_action_1(kf: float, s: float) -> float:
    return kf * s


def mass_action_2(kf: float, s1: float, s2: float) -> float:
    return kf * s1 * s2


def get_model() -> Model:
    p = {
        "kf_TPI": 1.0,
        "Keq_TPI": 21.0,
        "kf_Ald": 2000.0,
        "Keq_Ald": 7000.0,
    }
    p["kr_TPI"] = p["kf_TPI"] / p["Keq_TPI"]
    p["kr_Ald"] = p["kf_Ald"] / p["Keq_Ald"]

    GAP0 = 2.5e-5
    DHAP0 = GAP0 * p["Keq_TPI"]
    FBP0 = GAP0 * DHAP0 * p["Keq_Ald"]

    y0 = {"GAP": GAP0, "DHAP": DHAP0, "FBP": FBP0}

    return (
        Model()
        .add_variables(y0)
        .add_parameters(p)
        .add_reaction(
            "TPIf",
            mass_action_1,
            {"GAP": -1, "DHAP": 1},
            ["kf_TPI", "GAP"],
        )
        .add_reaction(
            "TPIr",
            mass_action_1,
            {"DHAP": -1, "GAP": 1},
            ["kr_TPI", "DHAP"],
        )
        .add_reaction(
            "ALDf",
            mass_action_2,
            {"DHAP": -1, "GAP": -1, "FBP": 1},
            ["kf_Ald", "DHAP", "GAP"],
        )
        .add_reaction(
            "ALDr",
            mass_action_1,
            {
                "FBP": -1,
                "DHAP": 1,
                "GAP": 1,
            },
            ["kr_Ald", "FBP"],
        )
    )

## Label Map

In [ ]:
mapper = LabelMapper(
    get_model(),
    label_variables={"GAP": 3, "DHAP": 3, "FBP": 6},
    label_maps={
        "TPIf": [2, 1, 0],
        "TPIr": [2, 1, 0],
        "ALDf": [0, 1, 2, 3, 4, 5],
        "ALDr": [0, 1, 2, 3, 4, 5],
    },
)

if (
    concs := Simulator(mapper.build_model(initial_labels={"GAP": 0}))
    .simulate(20)
    .get_full_concs()
) is not None:
    plot.relative_label_distribution(mapper, concs, n_cols=3)


## Linear Label Map

In [ ]:
m = get_model()

concs, fluxes = Simulator(m).simulate(20).get_concs_and_fluxes()
if concs is None or fluxes is None:
    raise ValueError

mapper = LinearLabelMapper(
    m,
    label_variables={"GAP": 3, "DHAP": 3, "FBP": 6},
    label_maps={
        "TPIf": [2, 1, 0],
        "TPIr": [2, 1, 0],
        "ALDf": [0, 1, 2, 3, 4, 5],
        "ALDr": [0, 1, 2, 3, 4, 5],
    },
)

if (
    concs := (
        Simulator(
            mapper.build_model(
                concs=concs.iloc[-1],
                fluxes=fluxes.iloc[-1],
                initial_labels={"GAP": 0},
            )
        )
        .simulate(20)
        .get_full_concs()
    )
) is not None:
    plot.relative_label_distribution(mapper, concs, n_cols=3)
